# Generation-based MRC 문제 풀기

In [ ]:
!nvidia-smi

## Requirements

* package
  * huggingface에서 제공하는 datasets과 transformers
  * sentencepiece : 단어를 나눌 때 활용함
  * nltk : 언어처리 관련 일반적인 tool

In [ ]:
!pip install datasets==1.4.1
!pip install transformers==4.4.1
!pip install sentencepiece==0.1.95
!pip install nltk

* tokenizer 다운로드

In [ ]:
import nltk
nltk.download('punkt')

## 데이터 및 평가 지표 불러오기

* 데이터셋 불러오기

In [ ]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

* EM과 F1 위주의 metric 가져오기

In [ ]:
from datasets import load_metric

metric = load_metric('squad')

## Pre-trained 모델 및 토크나이저 불러오기

* AutoModel은 사용하는 model에 따라 이름이 조금씩 다름

In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

* model 이름 정의
  * mt5-small
    * mt5 : multilingual t5
    * BART와 용도가 비슷함
    * 성능이나 디테일한 부분은 다름
    * seq-to-seq
    * text를 input으로 가져와서 text를 output으로 내는 generation model
    * small : 비교적 작은 사이즈 사용
    * vocabulary size가 아주 크고, 각 단어의 embedding을 저장해야하기 때문에 필요한 저장공간이 큼(약 1.2GB)
    * pre-trained language model을 활용하면 해당 구간의 언어만 활용하기 때문에 차지하는 용량이 줄어듬

In [ ]:
model_name = "google/mt5-small"

In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

In [ ]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config,
    cache_dir=None,
)

## 설정하기

* max_target_length
  * seq-to-seq이기 때문에 encoding 뿐만아니라 decoding을 해야함
  * decoding 단계에서 target length의 max를 정해줘야함

In [ ]:
max_source_length = 1024
max_target_length = 128
padding = False
preprocessing_num_workers=12
num_beams = 2
max_train_samples = 16
max_val_samples = 16
num_train_epochs = 3

## 전처리하기

* preprocess_function은 extraction-based에 비해 비교적 간단함
  * input과 target을 가져옴
  * tokenize함
  * tokenize한 값들을 model input에 해당하는 key로 지정함

In [ ]:
def preprocess_function(examples):
    inputs = [f'question: {q}  context: {c} </s>' for q, c in zip(examples['question'], examples['context'])]
    targets = [f'{a["text"][0]} </s>' for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # tokenize한 값들을 model input에 해당하는 key로 지정함
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["example_id"] = []
    for i in range(len(model_inputs["labels"])):
        model_inputs["example_id"].append(examples["id"][i])
    return model_inputs

In [ ]:
column_names = datasets['train'].column_names

* preprocess_function을 rambda 함수로 넣어줌으로서, 필요한 process를 효율적으로 진행함

* map을 활용하는 이유
  * num_worker를 활용하여 분산처리 하거나 thread 또는 process 별로 활용하는 등의 효율성을 높일 수 있기 때문
  * 효율성이 중요하지 않다면 직접 preprocess_function을 apply하는 방법으로 진행해도 됨

In [ ]:
train_dataset = datasets["train"]
train_dataset = train_dataset.select(range(max_train_samples))
train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )

In [ ]:
eval_dataset = datasets["validation"]
eval_dataset = eval_dataset.select(range(max_val_samples))
eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=False,
        )


## Fine-tuning 하기

* seq2seq을 위한 class 가져오기
  * DataCollatorForSeq2Seq : 다른 sequence length를 가지고 있는 input들을 합쳐주어 parallel computing을 하기 쉽게 만들어줌

In [ ]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

* label_pad_token_id
  * pad가 가지고 있는 token id
  * pad를 무시하는 경우 pad가 어떤 token id를 가지고 있는지 알아야함

In [ ]:
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=None,
        )

* postprocess_text
  * 예측값과 정답값을 postprocessing함

* compute_metrics
  * decode된 preds와 label을 가져와서 수치를 compute함

In [ ]:
def postprocess_text(preds, labels):
    # 공백 제거
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    
    # 문장별로 새로운 줄을 부여함
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # decoded_labels is for rouge metric, not used for f1/em metric
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    formatted_predictions = [{"id": ex['id'], "prediction_text": decoded_preds[i]} for i, ex in enumerate(datasets["validation"].select(range(max_val_samples)))]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"].select(range(max_val_samples))]

    result = metric.compute(predictions=formatted_predictions, references=references)
    return result

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir='outputs', 
    do_train=True, 
    do_eval=True, 
    predict_with_generate=True,
    num_train_epochs=num_train_epochs
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
train_result = trainer.train(resume_from_checkpoint=None)

In [ ]:
train_result

## 평가하기

* max_length
  * 중요함
* num_beams
  * beam search할 때 beam의 크기 정의

In [ ]:
metrics = trainer.evaluate(
    max_length=max_target_length, num_beams=num_beams, metric_key_prefix="eval"
)

In [ ]:
metrics

* generate()
  * output : 답이되는 id

In [ ]:
document = "세종대왕은 언제 태어났어?"
input_ids = tokenizer(document, return_tensors='pt').input_ids
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0], skip_special_tokens=True)